# Etapa 4: Otimização e Tuning de Hiperparâmetros

## Objetivos
Otimizar o melhor modelo da Etapa 3 através de tuning de hiperparâmetros, evitar overfitting, avaliar no conjunto de teste e salvar o modelo final otimizado.

**Variável Alvo**: `final_grade` (Nota Final)

**Lembre-se**: 
- Usar apenas treino e validação para tuning
- Avaliar no conjunto de teste APENAS UMA VEZ
- Comparar modelo antes vs depois da otimização
- Salvar modelo final para produção


In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import os
import warnings
from datetime import datetime

# Configuração para exibir gráficos inline no Jupyter Notebook
%matplotlib inline

# Configurações de visualização
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    plt.style.use('seaborn-darkgrid')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuração para exibir todas as colunas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Configuração adicional para garantir que gráficos apareçam
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

print("Bibliotecas importadas com sucesso!")
print(f"Data/Hora: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")


## 1. Recapitulação dos Resultados da Etapa 3

Carregar e revisar os resultados do modelo baseline da Etapa 3.


In [ ]:
# Carregar dados processados da Etapa 2
df = pd.read_csv('../../data/processed/students_clean.csv')

print("Dataset processado carregado com sucesso!")
print(f"\nFormato: {df.shape}")

# Separar features (X) e variável alvo (y)
target = 'final_grade'
features_to_remove = ['student_id', target]

X = df.drop(columns=features_to_remove, errors='ignore')
y = df[target]

print(f"\nFeatures (X): {X.shape}")
print(f"Target (y): {y.shape}")

# Carregar modelo baseline da Etapa 3
print("\n" + "="*60)
print("Carregando modelo baseline da Etapa 3...")
print("="*60)

try:
    modelo_baseline = joblib.load('../../models/modelo_baseline.pkl')
    print("✅ Modelo baseline carregado com sucesso!")
except FileNotFoundError:
    print("⚠️ Modelo baseline não encontrado. Será treinado novamente.")
    from sklearn.linear_model import LinearRegression
    modelo_baseline = LinearRegression()
    
    # Dividir dados para treinar baseline
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=True
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.25, random_state=42, shuffle=True
    )
    
    modelo_baseline.fit(X_train, y_train)
    print("✅ Modelo baseline treinado!")

# Dividir dados em treino, validação e teste (mesma divisão da Etapa 3)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, shuffle=True
)

print(f"\nDivisão dos Dados:")
print(f"Treino:      {X_train.shape[0]} amostras ({X_train.shape[0]/len(df)*100:.1f}%)")
print(f"Validação:   {X_val.shape[0]} amostras ({X_val.shape[0]/len(df)*100:.1f}%)")
print(f"Teste:       {X_test.shape[0]} amostras ({X_test.shape[0]/len(df)*100:.1f}%)")
print(f"\n⚠️ IMPORTANTE: Dados de teste serão usados APENAS UMA VEZ no final!")


In [ ]:
# Avaliar modelo baseline para comparação futura
print("="*60)
print("AVALIAÇÃO DO MODELO BASELINE (Etapa 3)")
print("="*60)

# Predições do baseline
y_train_pred_baseline = modelo_baseline.predict(X_train)
y_val_pred_baseline = modelo_baseline.predict(X_val)

# Métricas do baseline
mse_train_baseline = mean_squared_error(y_train, y_train_pred_baseline)
rmse_train_baseline = np.sqrt(mse_train_baseline)
mae_train_baseline = mean_absolute_error(y_train, y_train_pred_baseline)
r2_train_baseline = r2_score(y_train, y_train_pred_baseline)

mse_val_baseline = mean_squared_error(y_val, y_val_pred_baseline)
rmse_val_baseline = np.sqrt(mse_val_baseline)
mae_val_baseline = mean_absolute_error(y_val, y_val_pred_baseline)
r2_val_baseline = r2_score(y_val, y_val_pred_baseline)

# Criar DataFrame com resultados do baseline
resultados_baseline = pd.DataFrame({
    'Métrica': ['MSE', 'RMSE', 'MAE', 'R²'],
    'Treino': [mse_train_baseline, rmse_train_baseline, mae_train_baseline, r2_train_baseline],
    'Validação': [mse_val_baseline, rmse_val_baseline, mae_val_baseline, r2_val_baseline]
})

print("\nResultados do Modelo Baseline (Regressão Linear):")
print(resultados_baseline.to_string(index=False))

print(f"\n📊 Resumo:")
print(f"   R² Validação: {r2_val_baseline:.4f} ({r2_val_baseline*100:.2f}%)")
print(f"   RMSE Validação: {rmse_val_baseline:.4f} pontos")
print(f"   MAE Validação: {mae_val_baseline:.4f} pontos")


## 2. Seleção do Modelo para Otimização

Escolher o modelo que será otimizado. Como o baseline foi Regressão Linear, vamos otimizar modelos de regressão regularizada (Ridge, Lasso, ElasticNet) que são variações da Regressão Linear com regularização para evitar overfitting.


In [ ]:
# 2.1 Escolher modelo para otimização
# Vamos usar ElasticNet que combina Ridge e Lasso
# ElasticNet tem dois hiperparâmetros principais: alpha e l1_ratio

print("="*60)
print("SELEÇÃO DO MODELO PARA OTIMIZAÇÃO")
print("="*60)

print("\nModelo escolhido: ElasticNet")
print("\nJustificativa:")
print("  - ElasticNet combina regularização L1 (Lasso) e L2 (Ridge)")
print("  - Permite seleção de features (L1) e agrupamento de features correlacionadas (L2)")
print("  - Mais flexível que Ridge ou Lasso isolados")
print("  - Hiperparâmetros: alpha (força da regularização) e l1_ratio (proporção L1 vs L2)")

# Criar modelo base
modelo_base = ElasticNet(random_state=42, max_iter=10000)

print(f"\n✅ Modelo base criado: {type(modelo_base).__name__}")


## 3. Grid Search ou Random Search

Escolher uma técnica de otimização de hiperparâmetros. Vamos implementar ambas as opções, mas usar Random Search por ser mais eficiente.


In [ ]:
# 3.1 Definir grid de hiperparâmetros para ElasticNet
print("="*60)
print("DEFINIÇÃO DO GRID DE HIPERPARÂMETROS")
print("="*60)

# Hiperparâmetros para ElasticNet:
# - alpha: força da regularização (valores maiores = mais regularização)
# - l1_ratio: proporção de L1 (Lasso) vs L2 (Ridge)
#   - l1_ratio = 0: apenas Ridge (L2)
#   - l1_ratio = 1: apenas Lasso (L1)
#   - 0 < l1_ratio < 1: combinação (ElasticNet)

# Para Random Search, definimos distribuições
from scipy.stats import uniform, randint

# Distribuição para alpha (valores entre 0.01 e 100)
param_distributions = {
    'alpha': uniform(loc=0.01, scale=10),  # uniforme entre 0.01 e 10.01
    'l1_ratio': uniform(loc=0, scale=1)    # uniforme entre 0 e 1
}

print("\nHiperparâmetros para otimização (Random Search):")
print("  alpha: distribuição uniforme entre 0.01 e 10.01")
print("  l1_ratio: distribuição uniforme entre 0 e 1")
print("\nNúmero de iterações: 50 (pode ser aumentado para melhor resultado)")

# Para Grid Search (opção alternativa - comentado por ser mais lento)
# param_grid = {
#     'alpha': [0.01, 0.1, 1.0, 10.0, 100.0],
#     'l1_ratio': [0.0, 0.25, 0.5, 0.75, 1.0]
# }
# print(f"\nGrid Search teria {len(param_grid['alpha']) * len(param_grid['l1_ratio'])} combinações")


In [ ]:
# 3.2 Executar Random Search com Cross-Validation
print("="*60)
print("EXECUTANDO RANDOM SEARCH COM CROSS-VALIDATION")
print("="*60)
print("\n⏳ Isso pode levar alguns minutos...")
print("   Usando 5-fold cross-validation para cada combinação de hiperparâmetros")

# Combinar treino e validação para ter mais dados no tuning
X_train_val = pd.concat([X_train, X_val], axis=0)
y_train_val = pd.concat([y_train, y_val], axis=0)

print(f"\nDados combinados para tuning: {X_train_val.shape[0]} amostras")

# Criar RandomizedSearchCV
# n_iter: número de combinações aleatórias a testar
# cv: número de folds na cross-validation
# scoring: métrica para otimizar (neg_mean_squared_error = menor MSE)
# n_jobs: usar todos os cores disponíveis (-1)
# random_state: para reprodutibilidade

random_search = RandomizedSearchCV(
    estimator=modelo_base,
    param_distributions=param_distributions,
    n_iter=50,  # Testar 50 combinações aleatórias
    cv=5,  # 5-fold cross-validation
    scoring='neg_mean_squared_error',  # Queremos minimizar MSE
    n_jobs=-1,  # Usar todos os cores
    random_state=42,
    verbose=1  # Mostrar progresso
)

# Executar busca
import time
start_time = time.time()

random_search.fit(X_train_val, y_train_val)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\n✅ Random Search concluído em {elapsed_time:.2f} segundos ({elapsed_time/60:.2f} minutos)")
print(f"\nMelhores hiperparâmetros encontrados:")
print(f"  alpha: {random_search.best_params_['alpha']:.4f}")
print(f"  l1_ratio: {random_search.best_params_['l1_ratio']:.4f}")
print(f"\nMelhor score (neg MSE): {random_search.best_score_:.4f}")
print(f"Melhor RMSE (CV): {np.sqrt(-random_search.best_score_):.4f} pontos")


## 4. Análise dos Melhores Hiperparâmetros

Analisar os resultados do Random Search e entender quais hiperparâmetros funcionaram melhor.


In [ ]:
# 4.1 Converter resultados do Random Search em DataFrame
print("="*60)
print("ANÁLISE DOS RESULTADOS DO RANDOM SEARCH")
print("="*60)

# Converter cv_results_ em DataFrame
results_df = pd.DataFrame(random_search.cv_results_)

# Calcular RMSE a partir do neg_mean_squared_error
results_df['RMSE'] = np.sqrt(-results_df['mean_test_score'])
results_df['RMSE_std'] = np.sqrt(results_df['std_test_score'])

# Selecionar colunas relevantes
cols_to_show = ['param_alpha', 'param_l1_ratio', 'mean_test_score', 'RMSE', 'RMSE_std', 'rank_test_score']
results_analysis = results_df[cols_to_show].copy()

# Renomear colunas para melhor visualização
results_analysis.columns = ['Alpha', 'L1_Ratio', 'Neg_MSE', 'RMSE', 'RMSE_Std', 'Rank']

# Ordenar por rank (melhor = 1)
results_analysis = results_analysis.sort_values('Rank')

print("\nTop 10 Melhores Combinações de Hiperparâmetros:")
print("="*60)
print(results_analysis.head(10).to_string(index=False))


In [ ]:
# 4.2 Visualizar distribuição dos resultados
print("\nCriando visualizações dos resultados...")

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Gráfico 1: Alpha vs RMSE
ax1 = axes[0, 0]
scatter1 = ax1.scatter(results_analysis['Alpha'], results_analysis['RMSE'], 
                       c=results_analysis['L1_Ratio'], cmap='viridis', 
                       s=50, alpha=0.6, edgecolors='black')
ax1.set_xlabel('Alpha (Força da Regularização)', fontsize=11, fontweight='bold')
ax1.set_ylabel('RMSE (Cross-Validation)', fontsize=11, fontweight='bold')
ax1.set_title('Alpha vs RMSE\n(Cor = L1_Ratio)', fontsize=12, fontweight='bold')
ax1.grid(alpha=0.3)
plt.colorbar(scatter1, ax=ax1, label='L1_Ratio')

# Gráfico 2: L1_Ratio vs RMSE
ax2 = axes[0, 1]
scatter2 = ax2.scatter(results_analysis['L1_Ratio'], results_analysis['RMSE'],
                       c=results_analysis['Alpha'], cmap='plasma',
                       s=50, alpha=0.6, edgecolors='black')
ax2.set_xlabel('L1_Ratio (Proporção Lasso vs Ridge)', fontsize=11, fontweight='bold')
ax2.set_ylabel('RMSE (Cross-Validation)', fontsize=11, fontweight='bold')
ax2.set_title('L1_Ratio vs RMSE\n(Cor = Alpha)', fontsize=12, fontweight='bold')
ax2.grid(alpha=0.3)
plt.colorbar(scatter2, ax=ax2, label='Alpha')

# Gráfico 3: Distribuição do RMSE
ax3 = axes[1, 0]
ax3.hist(results_analysis['RMSE'], bins=20, edgecolor='black', alpha=0.7, color='skyblue')
ax3.axvline(results_analysis['RMSE'].min(), color='red', linestyle='--', 
            linewidth=2, label=f'Melhor: {results_analysis["RMSE"].min():.4f}')
ax3.axvline(results_analysis['RMSE'].mean(), color='green', linestyle='--',
            linewidth=2, label=f'Média: {results_analysis["RMSE"].mean():.4f}')
ax3.set_xlabel('RMSE', fontsize=11, fontweight='bold')
ax3.set_ylabel('Frequência', fontsize=11, fontweight='bold')
ax3.set_title('Distribuição do RMSE\n(Todas as Combinações Testadas)', fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(alpha=0.3)

# Gráfico 4: Top 10 melhores com barras de erro
ax4 = axes[1, 1]
top_10 = results_analysis.head(10)
x_pos = range(len(top_10))
bars = ax4.barh(x_pos, top_10['RMSE'], xerr=top_10['RMSE_Std'], 
                alpha=0.7, edgecolor='black', color='steelblue')
ax4.set_yticks(x_pos)
ax4.set_yticklabels([f"α={row['Alpha']:.3f}\nl1={row['L1_Ratio']:.3f}" 
                     for _, row in top_10.iterrows()], fontsize=9)
ax4.set_xlabel('RMSE (Cross-Validation)', fontsize=11, fontweight='bold')
ax4.set_ylabel('Combinação de Hiperparâmetros', fontsize=11, fontweight='bold')
ax4.set_title('Top 10 Melhores Combinações\n(com Desvio Padrão)', fontsize=12, fontweight='bold')
ax4.invert_yaxis()
ax4.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Visualizações criadas!")


In [ ]:
# 4.3 Análise dos melhores hiperparâmetros
print("\n" + "="*60)
print("INTERPRETAÇÃO DOS MELHORES HIPERPARÂMETROS")
print("="*60)

best_alpha = random_search.best_params_['alpha']
best_l1_ratio = random_search.best_params_['l1_ratio']

print(f"\nMelhor Alpha: {best_alpha:.4f}")
if best_alpha < 0.1:
    print("  → Regularização muito fraca (próximo de Regressão Linear simples)")
elif best_alpha < 1.0:
    print("  → Regularização moderada")
elif best_alpha < 10.0:
    print("  → Regularização forte")
else:
    print("  → Regularização muito forte")

print(f"\nMelhor L1_Ratio: {best_l1_ratio:.4f}")
if best_l1_ratio < 0.2:
    print("  → Predominância de Ridge (L2) - mantém todas as features")
elif best_l1_ratio < 0.5:
    print("  → Mais Ridge que Lasso, mas com alguma seleção de features")
elif best_l1_ratio < 0.8:
    print("  → Mais Lasso que Ridge - seleção de features importante")
else:
    print("  → Predominância de Lasso (L1) - seleção forte de features")

print(f"\n💡 Conclusão:")
print(f"   O modelo otimizado usa {best_l1_ratio*100:.1f}% de regularização L1 (Lasso)")
print(f"   e {(1-best_l1_ratio)*100:.1f}% de regularização L2 (Ridge)")
print(f"   com força de regularização alpha = {best_alpha:.4f}")


## 5. Treinamento do Modelo Final

Treinar o modelo final com os melhores hiperparâmetros usando TREINO + VALIDAÇÃO combinados.


In [ ]:
# 5.1 Treinar modelo final com melhores hiperparâmetros
print("="*60)
print("TREINAMENTO DO MODELO FINAL")
print("="*60)

# O best_estimator_ já está treinado com os melhores hiperparâmetros
# Mas vamos treinar novamente com todos os dados de treino+validação
# para ter o máximo de dados possível

print("\nTreinando modelo final com TREINO + VALIDAÇÃO combinados...")
print(f"Total de amostras: {X_train_val.shape[0]}")

# Criar modelo final com melhores hiperparâmetros
modelo_final = ElasticNet(
    alpha=random_search.best_params_['alpha'],
    l1_ratio=random_search.best_params_['l1_ratio'],
    random_state=42,
    max_iter=10000
)

# Treinar no conjunto completo (treino + validação)
modelo_final.fit(X_train_val, y_train_val)

print("✅ Modelo final treinado com sucesso!")

# Fazer predições para avaliação (ainda não usamos teste!)
y_train_val_pred = modelo_final.predict(X_train_val)

# Calcular métricas no conjunto de treino+validação
mse_train_val = mean_squared_error(y_train_val, y_train_val_pred)
rmse_train_val = np.sqrt(mse_train_val)
mae_train_val = mean_absolute_error(y_train_val, y_train_val_pred)
r2_train_val = r2_score(y_train_val, y_train_val_pred)

print(f"\nMétricas no conjunto de Treino+Validação:")
print(f"  R²:   {r2_train_val:.4f} ({r2_train_val*100:.2f}%)")
print(f"  RMSE: {rmse_train_val:.4f} pontos")
print(f"  MAE:  {mae_train_val:.4f} pontos")


## 6. Avaliação no Conjunto de Teste

⚠️ **IMPORTANTE**: Esta é a ÚNICA vez que usaremos o conjunto de teste!


In [ ]:
# 6.1 Carregar dados de teste (guardados desde a Etapa 3)
print("="*60)
print("AVALIAÇÃO FINAL NO CONJUNTO DE TESTE")
print("="*60)
print("\n⚠️ ATENÇÃO: Esta é a ÚNICA vez que usaremos o conjunto de teste!")

# Carregar dados de teste salvos
X_test = pd.read_csv('../../data/processed/X_test.csv')
y_test = pd.read_csv('../../data/processed/y_test.csv').squeeze()

print(f"\nDados de teste carregados:")
print(f"  X_test: {X_test.shape}")
print(f"  y_test: {y_test.shape}")

# Fazer predições no conjunto de teste
print("\nFazendo predições no conjunto de teste...")
y_test_pred = modelo_final.predict(X_test)

# Calcular métricas finais
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

print("\n" + "="*60)
print("MÉTRICAS FINAIS NO CONJUNTO DE TESTE")
print("="*60)
print(f"\nMSE  (Erro Quadrático Médio):     {mse_test:.4f}")
print(f"RMSE (Raiz do Erro Quadrático):    {rmse_test:.4f} pontos")
print(f"MAE  (Erro Absoluto Médio):        {mae_test:.4f} pontos")
print(f"R²   (Coeficiente de Determinação): {r2_test:.4f} ({r2_test*100:.2f}%)")

print("\n💡 Interpretação:")
print(f"   O modelo explica {r2_test*100:.2f}% da variação nas notas finais")
print(f"   Em média, o modelo erra por {rmse_test:.2f} pontos")
print(f"   O erro médio absoluto é de {mae_test:.2f} pontos")


## 7. Comparação: Antes vs Depois da Otimização

Comparar o desempenho do modelo baseline (Etapa 3) com o modelo otimizado (Etapa 4).


In [ ]:
# 7.1 Avaliar baseline no conjunto de teste para comparação justa
print("="*60)
print("COMPARAÇÃO: BASELINE vs MODELO OTIMIZADO")
print("="*60)

# Avaliar baseline no teste (para comparação justa)
y_test_pred_baseline = modelo_baseline.predict(X_test)

mse_test_baseline = mean_squared_error(y_test, y_test_pred_baseline)
rmse_test_baseline = np.sqrt(mse_test_baseline)
mae_test_baseline = mean_absolute_error(y_test, y_test_pred_baseline)
r2_test_baseline = r2_score(y_test, y_test_pred_baseline)

# Criar tabela comparativa
comparacao = pd.DataFrame({
    'Métrica': ['MSE', 'RMSE', 'MAE', 'R²'],
    'Baseline (Etapa 3)': [
        mse_test_baseline,
        rmse_test_baseline,
        mae_test_baseline,
        r2_test_baseline
    ],
    'Otimizado (Etapa 4)': [
        mse_test,
        rmse_test,
        mae_test,
        r2_test
    ]
})

# Calcular melhoria
comparacao['Melhoria'] = comparacao['Baseline (Etapa 3)'] - comparacao['Otimizado (Etapa 4)']
comparacao['Melhoria (%)'] = (
    (comparacao['Melhoria'] / comparacao['Baseline (Etapa 3)']) * 100
).round(2)

# Para R², queremos aumento (não diminuição)
comparacao.loc[comparacao['Métrica'] == 'R²', 'Melhoria (%)'] = (
    ((comparacao.loc[comparacao['Métrica'] == 'R²', 'Otimizado (Etapa 4)'].values[0] - 
      comparacao.loc[comparacao['Métrica'] == 'R²', 'Baseline (Etapa 3)'].values[0]) / 
     comparacao.loc[comparacao['Métrica'] == 'R²', 'Baseline (Etapa 3)'].values[0]) * 100
).round(2)

print("\nComparação no Conjunto de Teste:")
print("="*60)
print(comparacao.to_string(index=False))


In [ ]:
# 7.2 Visualizar comparação
print("\nCriando gráficos de comparação...")

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Comparação de métricas
ax1 = axes[0]
x = np.arange(len(comparacao['Métrica']))
width = 0.35

bars1 = ax1.bar(x - width/2, comparacao['Baseline (Etapa 3)'], width, 
                label='Baseline', alpha=0.8, color='lightcoral', edgecolor='black')
bars2 = ax1.bar(x + width/2, comparacao['Otimizado (Etapa 4)'], width,
                label='Otimizado', alpha=0.8, color='steelblue', edgecolor='black')

ax1.set_xlabel('Métrica', fontsize=12, fontweight='bold')
ax1.set_ylabel('Valor', fontsize=12, fontweight='bold')
ax1.set_title('Comparação: Baseline vs Modelo Otimizado\n(Conjunto de Teste)', 
              fontsize=13, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(comparacao['Métrica'])
ax1.legend(fontsize=10)
ax1.grid(axis='y', alpha=0.3)

# Adicionar valores nas barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

# Gráfico 2: Melhoria percentual
ax2 = axes[1]
colors = ['green' if x > 0 else 'red' for x in comparacao['Melhoria (%)']]
bars = ax2.barh(comparacao['Métrica'], comparacao['Melhoria (%)'], 
                color=colors, alpha=0.7, edgecolor='black')
ax2.axvline(x=0, color='black', linestyle='-', linewidth=1)
ax2.set_xlabel('Melhoria (%)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Métrica', fontsize=12, fontweight='bold')
ax2.set_title('Melhoria Percentual do Modelo Otimizado\n(Verde = Melhorou | Vermelho = Piorou)', 
              fontsize=13, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

# Adicionar valores nas barras
for bar in bars:
    width = bar.get_width()
    label_x = width if width > 0 else width - 0.5
    ax2.text(label_x, bar.get_y() + bar.get_height()/2,
            f' {width:.2f}%', ha='left' if width > 0 else 'right', 
            va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("✅ Gráficos de comparação criados!")

# Resumo da melhoria
print("\n" + "="*60)
print("RESUMO DA MELHORIA")
print("="*60)
melhoria_r2 = comparacao.loc[comparacao['Métrica'] == 'R²', 'Melhoria (%)'].values[0]
melhoria_rmse = comparacao.loc[comparacao['Métrica'] == 'RMSE', 'Melhoria (%)'].values[0]

if melhoria_r2 > 0:
    print(f"\n✅ R² melhorou em {melhoria_r2:.2f}%")
else:
    print(f"\n⚠️ R² diminuiu em {abs(melhoria_r2):.2f}%")

if melhoria_rmse < 0:  # RMSE menor é melhor
    print(f"✅ RMSE melhorou em {abs(melhoria_rmse):.2f}% (diminuiu)")
else:
    print(f"⚠️ RMSE piorou em {melhoria_rmse:.2f}% (aumentou)")


## 8. Análise de Erros Detalhada

Realizar análise detalhada dos erros do modelo final no conjunto de teste.


In [ ]:
# 8.1 Scatter Plot: Predito vs Real
print("="*60)
print("ANÁLISE DE ERROS DETALHADA")
print("="*60)

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Gráfico 1: Predito vs Real
ax1 = axes[0]
ax1.scatter(y_test, y_test_pred, alpha=0.5, s=30, color='steelblue', edgecolors='black', linewidth=0.5)

# Linha perfeita (y = x)
min_val = min(y_test.min(), y_test_pred.min())
max_val = max(y_test.max(), y_test_pred.max())
ax1.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Linha Perfeita (Predição = Real)')

ax1.set_xlabel('Valores Reais (Nota Final)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Predições do Modelo (Nota Final)', fontsize=12, fontweight='bold')
ax1.set_title('Scatter Plot: Predições vs Valores Reais\n(Conjunto de Teste)', 
              fontsize=13, fontweight='bold')
ax1.legend(fontsize=10, loc='lower right')
ax1.grid(alpha=0.3)

# Adicionar R² e RMSE no gráfico
textstr = f'R² = {r2_test:.4f} ({r2_test*100:.2f}%)\nRMSE = {rmse_test:.2f} pontos'
ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes,
         fontsize=11, verticalalignment='top', 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

# Gráfico 2: Resíduos vs Predições
residuos = y_test - y_test_pred
ax2 = axes[1]
ax2.scatter(y_test_pred, residuos, alpha=0.5, s=30, color='steelblue', edgecolors='black', linewidth=0.5)
ax2.axhline(y=0, color='red', linestyle='--', linewidth=2, label='Zero (Ideal)')
ax2.set_xlabel('Predições do Modelo (Nota Final)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Resíduos (Valor Real - Predição)', fontsize=12, fontweight='bold')
ax2.set_title('Resíduos vs Predições\n(Verificar Padrões e Homocedasticidade)', 
              fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Gráficos de análise de erros criados!")


In [ ]:
# 8.2 Distribuição dos Resíduos
print("\nCriando gráficos de distribuição dos resíduos...")

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Subplot 1: Histograma dos resíduos
ax1 = axes[0]
ax1.hist(residuos, bins=30, edgecolor='black', alpha=0.7, color='skyblue', label='Distribuição dos Resíduos')
ax1.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero (Ideal)')
ax1.axvline(x=residuos.mean(), color='green', linestyle='--', linewidth=2, 
            label=f'Média: {residuos.mean():.2f}')
ax1.set_xlabel('Resíduos (Valor Real - Predição)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Frequência (Quantidade de Amostras)', fontsize=12, fontweight='bold')
ax1.set_title('Histograma: Distribuição dos Resíduos\n(Conjunto de Teste)', 
              fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(axis='y', alpha=0.3)

# Adicionar estatísticas
textstr = f'Média: {residuos.mean():.4f}\nDesvio Padrão: {residuos.std():.4f}\nMínimo: {residuos.min():.2f}\nMáximo: {residuos.max():.2f}'
ax1.text(0.98, 0.98, textstr, transform=ax1.transAxes,
         fontsize=10, verticalalignment='top', horizontalalignment='right',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

# Subplot 2: Q-Q Plot (verificar normalidade dos resíduos)
from scipy import stats
ax2 = axes[1]
stats.probplot(residuos, dist="norm", plot=ax2)
ax2.set_title('Q-Q Plot: Normalidade dos Resíduos\n(Conjunto de Teste)', 
              fontsize=13, fontweight='bold')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Estatísticas dos resíduos
print("\nEstatísticas dos Resíduos:")
print("="*60)
print(f"  Média: {residuos.mean():.4f} (ideal: próxima de 0)")
print(f"  Desvio Padrão: {residuos.std():.4f}")
print(f"  Mínimo: {residuos.min():.4f}")
print(f"  Máximo: {residuos.max():.4f}")
print(f"  Mediana: {residuos.median():.4f}")

print("\n💡 Interpretação:")
if abs(residuos.mean()) < 0.5:
    print("  ✅ Média próxima de zero: modelo não tem viés sistemático")
else:
    print("  ⚠️ Média distante de zero: modelo pode ter viés")

if abs(residuos.skew()) < 0.5:
    print("  ✅ Distribuição aproximadamente simétrica")
else:
    print(f"  ⚠️ Distribuição assimétrica (skewness: {residuos.skew():.2f})")


In [ ]:
# 8.3 Análise de Casos Extremos (Piores Predições)
print("\n" + "="*60)
print("ANÁLISE DE CASOS EXTREMOS - PIORES PREDIÇÕES")
print("="*60)

# Calcular erro absoluto
erro_absoluto = np.abs(residuos)

# Encontrar top 10 piores predições
top_10_erros = erro_absoluto.nlargest(10)

# Criar DataFrame com informações dos piores casos
casos_extremos = pd.DataFrame({
    'Índice': top_10_erros.index,
    'Valor Real': y_test.iloc[top_10_erros.index],
    'Predição': y_test_pred[top_10_erros.index],
    'Erro Absoluto': top_10_erros.values,
    'Resíduo': residuos.iloc[top_10_erros.index]
})

# Ordenar por erro absoluto (maior primeiro)
casos_extremos = casos_extremos.sort_values('Erro Absoluto', ascending=False)

print("\nTop 10 Piores Predições (Maiores Erros):")
print("="*60)
print(casos_extremos.to_string(index=False))

# Análise
print("\n💡 Análise dos Casos Extremos:")
print(f"   Maior erro: {casos_extremos['Erro Absoluto'].max():.2f} pontos")
print(f"   Erro médio nos top 10: {casos_extremos['Erro Absoluto'].mean():.2f} pontos")
print(f"   Erro médio geral: {erro_absoluto.mean():.2f} pontos")

# Verificar se há padrão (subestimação ou superestimação)
subestimados = (casos_extremos['Resíduo'] > 0).sum()
superestimados = (casos_extremos['Resíduo'] < 0).sum()

print(f"\n   Nos piores casos:")
print(f"   - Subestimados (predição < real): {subestimados} casos")
print(f"   - Superestimados (predição > real): {superestimados} casos")


## 9. Salvamento do Modelo Final

Salvar o modelo otimizado e treinado para uso em produção.


In [ ]:
# 9.1 Salvar o modelo final otimizado
print("="*60)
print("SALVAMENTO DO MODELO FINAL")
print("="*60)

# Criar diretório models se não existir
os.makedirs('../../models', exist_ok=True)

# Salvar modelo final
model_path = '../../models/modelo_final.pkl'
joblib.dump(modelo_final, model_path)

print(f"✅ Modelo final salvo em: {model_path}")
print(f"\nInformações do Modelo:")
print(f"   Tipo: ElasticNet")
print(f"   Alpha: {random_search.best_params_['alpha']:.4f}")
print(f"   L1_Ratio: {random_search.best_params_['l1_ratio']:.4f}")
print(f"   Features: {X_train_val.shape[1]}")
print(f"\nMétricas no Conjunto de Teste:")
print(f"   R²:   {r2_test:.4f} ({r2_test*100:.2f}%)")
print(f"   RMSE: {rmse_test:.4f} pontos")
print(f"   MAE:  {mae_test:.4f} pontos")

# Verificar tamanho do arquivo
if os.path.exists(model_path):
    file_size = os.path.getsize(model_path) / 1024  # KB
    print(f"\n   Tamanho do arquivo: {file_size:.2f} KB")


In [ ]:
# 9.2 Testar se o modelo carregado funciona corretamente
print("\nTestando modelo carregado...")
print("="*60)

# Carregar modelo
modelo_carregado = joblib.load(model_path)

# Fazer predição de teste
predicao_teste = modelo_carregado.predict(X_test.iloc[:5])

print("✅ Modelo carregado com sucesso!")
print(f"\nTeste de predição (primeiras 5 amostras):")
print(f"  Valores reais:    {y_test.iloc[:5].values}")
print(f"  Predições:        {predicao_teste}")
print(f"  Erros:            {y_test.iloc[:5].values - predicao_teste}")

# Verificar se as predições são iguais
predicoes_originais = modelo_final.predict(X_test.iloc[:5])
if np.allclose(predicao_teste, predicoes_originais):
    print("\n✅ Modelo carregado funciona corretamente!")
else:
    print("\n⚠️ Diferenças detectadas entre modelo original e carregado")


## 10. Conclusões Finais

Resumo final dos resultados da otimização e próximos passos.


In [ ]:
# Resumo final
print("="*80)
print("RESUMO FINAL - ETAPA 4: OTIMIZAÇÃO E TUNING DE HIPERPARÂMETROS")
print("="*80)

print(f"\n📊 Modelo Otimizado:")
print(f"   Tipo: ElasticNet")
print(f"   Alpha: {random_search.best_params_['alpha']:.4f}")
print(f"   L1_Ratio: {random_search.best_params_['l1_ratio']:.4f}")

print(f"\n📈 Métricas de Desempenho (Conjunto de Teste):")
print(f"   R²:   {r2_test:.4f} ({r2_test*100:.2f}%)")
print(f"   RMSE: {rmse_test:.4f} pontos")
print(f"   MAE:  {mae_test:.4f} pontos")

print(f"\n🔄 Comparação com Baseline:")
melhoria_r2 = comparacao.loc[comparacao['Métrica'] == 'R²', 'Melhoria (%)'].values[0]
melhoria_rmse = comparacao.loc[comparacao['Métrica'] == 'RMSE', 'Melhoria (%)'].values[0]
print(f"   R²:   {melhoria_r2:+.2f}%")
print(f"   RMSE: {melhoria_rmse:+.2f}%")

print(f"\n🔍 Análise de Erros:")
print(f"   Média dos resíduos: {residuos.mean():.4f}")
print(f"   Desvio padrão: {residuos.std():.4f}")
print(f"   Maior erro: {erro_absoluto.max():.2f} pontos")

print(f"\n💾 Arquivos Salvos:")
print(f"   Modelo final: ../../models/modelo_final.pkl")

print("\n" + "="*80)
print("ETAPA 4 CONCLUÍDA!")
print("="*80)


### Documentação: Storytelling e Conclusões

#### Contexto
[Descrever o objetivo da otimização e o que foi feito]

#### Metodologia de Otimização
[Explicar por que escolheu Random Search vs Grid Search]
[Explicar por que escolheu ElasticNet]
[Descrever o processo de cross-validation]

#### Resultados da Otimização
[Interpretar os melhores hiperparâmetros encontrados]
[Explicar o que significam alpha e l1_ratio escolhidos]

#### Comparação: Antes vs Depois
[Analisar se houve melhoria significativa]
[Explicar por que melhorou (ou não melhorou)]

#### Análise de Erros
[Interpretar os gráficos de resíduos]
[Analisar os casos extremos - por que o modelo errou?]
[Verificar se há padrões nos erros]

#### Limitações do Modelo
[Quais são as limitações do modelo atual?]
[Em que situações o modelo pode falhar?]

#### Possíveis Melhorias Futuras
[O que poderia ser feito para melhorar ainda mais?]
[Quais outras técnicas poderiam ser testadas?]
[Feature engineering adicional? Outros algoritmos?]

#### Conclusões
[Resumir o desempenho final do modelo]
[O modelo está pronto para produção?]
[Quais são os próximos passos?]
